In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from single_photons.environment import Particle
from scipy import signal as sn
from scipy.optimize import curve_fit


In [ ]:
omega = 2*np.pi*1e5
T = 2*np.pi/omega
t = np.arange(0, 1000*T, T/400)
N = t.shape[0]
delta_t = np.diff(t)[0]

In [4]:
gamma = 32000
env = Particle(omega, gamma)

TypeError: __init__() missing 1 required positional argument: 'coupling'

In [ ]:
std_detection = 1

In [ ]:
states = np.array([[1],[0.]])
new_states = np.zeros((N,2)) 
measured_states = np.zeros((N, 2))
for i in tqdm(range(t.shape[0])):
    new_states[i,:] = states[:,0]
    measured_states[i, :] = states[:, 0] + std_detection*np.random.normal()
    states = env.step(states, control=0, delta_t=delta_t)

In [ ]:
fig = plt.Figure()
plt.title('Position')
plt.plot(t, measured_states[:,0])
plt.plot(t, new_states[:,0])
plt.grid()
plt.legend(['Measured', 'Simulated'])
plt.show()

In [ ]:
fig = plt.Figure()
plt.title('Velocity')
plt.plot(t, measured_states[:,1])
plt.plot(t, new_states[:,1])
plt.grid()
plt.legend(['Measured', 'Simulated'])
plt.show()

In [ ]:
freq, pxx = sn.periodogram(new_states[:,0], 1/delta_t)

In [ ]:
plt.semilogy(freq, pxx)
plt.xscale('log')
plt.xlim([0, 3e6])
plt.ylim([1e-12, 4*max(pxx)])